<a href="https://colab.research.google.com/github/skylerputney/MachineLearning/blob/main/Dijkstra_CityPathSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Define and Populate a Data Structure
The following defines a data structure (graph representation) containing the cities and their inner-city links and cost (road distance in miles) for the Romania example shown in Figure 3.1 on page 64 of *Artificial Intelligence A Modern Approach Fourth Edition* by Stuart Russell and Peter Norvig.

The data structure is formatted as a dictionary of dictionaries such that the key of the outer dictionary is a city and its value is a dictionary containing neighboring cities (keys) and the traversal cost (values).

In [ ]:
romania_map = {
    'Neamt': {'Iasi': 87},
    'Iasi': {'Neamt': 87, 'Vaslui': 92},
    'Vaslui': {'Iasi': 92, 'Urziceni': 142},
    'Urziceni': {'Vaslui': 142, 'Hirsova': 98, 'Bucharest': 85},
    'Hirsova': {'Urziceni': 98, 'Eforie': 86},
    'Eforie': {'Hirsova': 86},
    'Bucharest': {'Urziceni': 85, 'Giurgiu': 90, 'Pitesti': 101, 'Fagaras': 211},
    'Giurgiu': {'Bucharest': 90},
    'Pitesti': {'Bucharest': 101, 'Craiova': 138, 'Rimnicu Vilcea': 97},
    'Fagaras': {'Bucharest': 211, 'Sibiu': 99},
    'Craiova': {'Pitesti': 138, 'Rimnicu Vilcea': 146, 'Drobeta': 120},
    'Rimnicu Vilcea': {'Pitesti': 97, 'Sibiu': 80, 'Craiova': 146},
    'Sibiu': {'Fagaras': 99, 'Rimnicu Vilcea': 80, 'Arad': 140, 'Oradea': 151},
    'Drobeta': {'Craiova': 120, 'Mehadia': 75},
    'Mehadia': {'Drobeta': 75, 'Lugoj': 70},
    'Lugoj': {'Mehadia': 75, 'Timisoara': 111},
    'Timisoara': {'Lugoj': 111, 'Arad': 118},
    'Arad': {'Timisoara': 118, 'Sibiu': 140, 'Zerind': 75},
    'Zerind': {'Arad': 75, 'Oradea': 71},
    'Oradea': {'Zerind': 71, 'Sibiu': 151}
}

#Define a Node Data Structure
The following defines a Node data structure for use by the Uniform Cost Search Algorithm

The Node data structure features the following attributes:

*   state: The state to which the node corresponds (city)
*   parent: The node which generated this node (previous city)
*   path_cost: Cumulative cost to reach this node from initial state

The parent defaults to None and path_cost to 0 if no value is given, representing the initial state.

The Node data structure has the following functions:
*   get_path(self): Reconstructs the path from the initial state to this node
*   \_\_lt\_\_(self, other): Defines comparison method for Node objects based on path_cost so that a PriorityQueue can maintain nodes in ascending path_cost order



In [ ]:
class Node:
  def __init__(self, state, parent=None, path_cost=0):
    self.state = state
    self.parent = parent
    self.path_cost = path_cost

  def get_path(self):
    path = []
    node = self
    while node is not None:  # Loop through until reaching initial state
      path.append(node.state)  # Append current state to path
      node = node.parent  # Retrieve parent for next loop iteration
    return path[::-1]  # Reverse path to get in-order from initial state to current state

  def __lt__(self, other):
    return self.path_cost < other.path_cost  # Let PriorityQueue compare nodes based on path_cost


#Display Routes Algorithm Explores
Formats and prints the Initialization or Given Iteration's Frontier Queue by:
1. Converting the PriorityQueue to a list to make it iterable and sorting by ascending path_cost to get an in-order print with lowest-cost-path first.
2. Checking if the iteration is 0, and if so, printing the Initialization message formatted as: { [ start_state , 0 ] } before returning
3. Joining nodes' paths (cities traversed) with '->' and formatting them and the traversal cost as: [ path , cost ] where path is like: A->B->...
4. Joining prior formatted paths and associated costs with ' , '
5. Formatting a resultant Iteration string as: IterationX: { prior_joined_formatted_paths_and_costs }
6. Printing the resultant Iteration string

In [ ]:
def print_frontier(frontier_queue, iteration):
  frontier_list = sorted(list(frontier_queue.queue), key=lambda node: node.path_cost)  # Convert PriorityQueue to list to make iterable, sort by ascending cost
  if iteration == 0:
      print(f"Initialization: {{ [ {frontier_list[0].state} , 0 ] }}")  # Print Initialization, formatted like : { [ Initial_State , 0 ]}
      return  # Prevent printing in Iteration format
  formatted_frontier_queue = [f"[ {'->'.join(node.get_path())} , {node.path_cost} ]" for node in frontier_list]  # Format each path as: [ A->B->... , cost ]
  formatted_frontier_queue_string = ' , '.join(formatted_frontier_queue)  # Format current iteration's paths as: { [ A->B->... , cost] , [ A->B->... , cost] , ... }
  formatted_iteration_string = f"Iteration{iteration}: {{ {formatted_frontier_queue_string} }}"  # Format iteration as: IterationX: { [ A->B->... , cost] , [ A->B->... , cost] , ... }
  print(formatted_iteration_string)  # Print formatted iteration string

#Implementation of the Uniform-Cost Search Algorithm (Dijkstra's Algorithm)
General-form implementation of the uniform-cost search algorithm based on a best-first-search algorithm with the evaluation function being path_cost.

Inputs:
* Graph defined as a dictionary containing entries: State: {NeighborState1: Cost, NeighborState2: Cost, ...}, ...
* Initial (starting) State
* Destination (goal) State

Prints:
* Frontier Queue with paths and costs at Initialization and each Iteration, excepting the last
* Solution with best path on last Iteration
* No Path Found message if algorithm fails to find a valid path

Output:
* Solution Node, or None in event of failure


In [ ]:
import queue

def uniform_cost_search(graph, start, dest):
  frontier = queue.PriorityQueue()  # Nodes that have been discovered but not yet explored, maintained in lowest-cost-first order
  initial_node = Node(start)  # Node for initial (starting) state
  frontier.put(initial_node)  # Initialize frontier with initial Node
  reached = {start: initial_node}  # Nodes that have been generated (could or could not be yet expanded)
  iteration = 0  # Track current iteration

  while not frontier.empty():  # Search until frontier queue is empty
    print_frontier(frontier, iteration)  # Print frontier from previous iteration
    iteration += 1  # Increment iteration count
    current_node = frontier.get()  # Obtain node with lowest path_cost

    if current_node.state == dest:  # Destination (goal) has been reached
      print(f"Solution: Iteration{iteration} gives the final output as {'->'.join(current_node.get_path())}.")  # Print solution (best route from origin to destination)
      return current_node  # Return solution node

    # Expand the current_node, add its unreached neighbors to frontier or replace reached neighbor if lower cost path is found
    for neighbor_node_state, traversal_cost in graph[current_node.state].items():  # Get neighbor node's state and traversal cost from the current_node from graph
      new_cost = current_node.path_cost + traversal_cost  # Obtain total traversal cost to neighbor node from initial state
      if neighbor_node_state not in reached or new_cost < reached[neighbor_node_state].path_cost:  # If neighbor node has not been generated or a lower cost path is found
        neighbor_node = Node(neighbor_node_state, current_node, new_cost)  # Generate neighbor node
        reached[neighbor_node_state] = neighbor_node  # Mark neighbor node as generated
        frontier.put(neighbor_node)  # Add neighbor node to frontier

  # Failed to find a path
  print(f'No path found from {start} to {dest}.')
  return None


#Prompt User for City of Origin and Destination City and Run the Algorithm
Obtains an origin and destination city from the user and calls the Uniform-Cost Search Algorithm.

Strips any leading/trailing whitespace from the gathered input.

Inputs are case-sensitive according to the defined romania_map:
*   First letter of each word in the city name must be capitalized and all other characters lowercase

If an invalid city name is entered, an AssertionError is raised, an error message printed, and execution stops.

In [ ]:
origin = input("Please enter the city of origin from the Romania map: ").strip()  # Gather origin city input, strip whitespace
assert origin in romania_map, f"{origin} not found in Romania map. Please enter a valid origin city."
destination = input("Please enter the destination city from the Romania map: ").strip()  # Gather destination city input, strip whitespace
assert destination in romania_map, f"{destination} not found in Romania map. Please enter a valid destination city."

_ = uniform_cost_search(romania_map, origin, destination)  # Run the algorithm, discard returned result node to prevent printing

Please enter the city of origin from the Romania map: Bucharest
Please enter the destination city from the Romania map: Rimnicu Vilcea
Initialization: { [ Bucharest , 0 ] }
Iteration1: { [ Bucharest->Urziceni , 85 ] , [ Bucharest->Giurgiu , 90 ] , [ Bucharest->Pitesti , 101 ] , [ Bucharest->Fagaras , 211 ] }
Iteration2: { [ Bucharest->Giurgiu , 90 ] , [ Bucharest->Pitesti , 101 ] , [ Bucharest->Urziceni->Hirsova , 183 ] , [ Bucharest->Fagaras , 211 ] , [ Bucharest->Urziceni->Vaslui , 227 ] }
Iteration3: { [ Bucharest->Pitesti , 101 ] , [ Bucharest->Urziceni->Hirsova , 183 ] , [ Bucharest->Fagaras , 211 ] , [ Bucharest->Urziceni->Vaslui , 227 ] }
Iteration4: { [ Bucharest->Urziceni->Hirsova , 183 ] , [ Bucharest->Pitesti->Rimnicu Vilcea , 198 ] , [ Bucharest->Fagaras , 211 ] , [ Bucharest->Urziceni->Vaslui , 227 ] , [ Bucharest->Pitesti->Craiova , 239 ] }
Iteration5: { [ Bucharest->Pitesti->Rimnicu Vilcea , 198 ] , [ Bucharest->Fagaras , 211 ] , [ Bucharest->Urziceni->Vaslui , 227 ] , [